In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
seed = 11667
n_collect = 200
cvs = np.concatenate([np.linspace(0, 0.2, 11), np.linspace(0.2, 0.5, 13)[1:]])
coverage0 = 0.2

folders = [
    'dy_mul', 
    'dy_add',
    'dy_ladd'
]

In [3]:
def collect_sens(pars_post, pars_sel, changes=[-0.1, -0.05, 0, 0.05, 0.1], coverage=0.2, alg='VSC'):    
    m = ModelIntv()

    intv = {'FullACF': {'Coverage': coverage, 'ScreenAlg': alg}}
    
    mss_stats = list()
    
    for p in pars_sel:
        print(p)
        for i, pars in enumerate(pars_post):
            for change in changes:
                ps = dict(pars)
                ps[p] = ps[p] * (1 + change)               
                
                y1, ps = m.find_baseline(ps, 2022)
                
                _, (_, ms_stats0), _ = m.simulate_onward(y1, ps, intv={}, polished=True)
                _, (_, ms_stats1), _ = m.simulate_onward(y1, ps, intv=intv, polished=True)
                ms_stats = {
                    'Inc': 1 - ms_stats1['Inc'] / ms_stats0['Inc'],
                    'Mor': 1 - ms_stats1['Mor'] / ms_stats0['Mor'],
                    'Key': i, 'Change': change, 'Par': p
                }
        
                mss_stats.append(ms_stats)

    mss_stats = pd.DataFrame(mss_stats).assign(Alg=alg, Coverage=coverage)
    return mss_stats

In [4]:
pars_sel = [
    'beta_ds',
    'rr_beta_dr', 'rr_inf_asym', 'rr_sus_ltbi',
    'r_react', 'r_relapse_td', 'r_relapse_tc', 'r_relapse_st',
    'r_onset', 'r_clear',
    'r_cs_s', 'r_cs_c',
    'p_entry_pri', 'p_dx_pub', 'p_dx_pri'
]

In [ ]:
for folder in folders:    
    rd.seed(seed)
    
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))

    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    pars_post = [pars_post[i] for i in sel]

    mss_stats = collect_sens(pars_post, pars_sel, 
                             changes=[-0.1, -0.05, 0, 0.05, 0.1], 
                             coverage=0.2, alg='VSC')
    
    mss_stats.to_csv(f'{out_path}/Sens_Pars_stats.csv')

dy_mul
beta_ds
